# Lab 1: Image Classification

### Task 1: Preprocessing and Data Augmentation

* **STEP 1:** Import required libraries

In [ ]:
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers, optimizers, applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import warnings
warnings.filterwarnings("ignore")
warnings.warn("this will not show")

In [ ]:
# Data augmentation
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    keras.layers.RandomRotation(factor=0.2),
    layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
    layers.RandomZoom(height_factor=(0.2, 0.3), width_factor=(0.2, 0.3)),
    layers.RandomBrightness(factor=0.1),
    layers.RandomContrast(factor=0.1)
])

In [ ]:
datagenerator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # Assuming you want to split 20% for validation
)

In [ ]:
batch_size = 32
train_generator = datagenerator.flow_from_directory(
    "FlowersDataset//train",
    target_size=(300,300),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

Found 2198 images belonging to 5 classes.


In [ ]:
valid_generator = datagenerator.flow_from_directory(
    "FlowersDataset//train",
    target_size=(300, 300),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 548 images belonging to 5 classes.


In [ ]:
# Customizing the model based on InceptionV3
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense

In [ ]:
# Initializing InceptionV3 (pretrained) model with input image shape as (300, 300, 3)
base_model = InceptionV3(weights=None, include_top=False, input_shape=(300, 300, 3))

In [ ]:
base_model.load_weights('\\kaggle\\input\\inceptionv3\\inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')
# Setting the Training of all layers of InceptionV3 model to false
base_model.trainable = False


# Draft

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
# from keras import ops
from tensorflow.keras import models, layers
from tensorflow.python.keras.layers import Dense

In [ ]:
from keras.models import Sequential
from keras.layers import Activation, Dense

In [ ]:
# Load the pre-trained model from TensorFlow Hub
num_classes = 5
module_url = "https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/5"

In [ ]:
base_model=hub.KerasLayer(module_url, trainable=False)

In [ ]:
model = Sequential()

#keras.layers.Dense(num_classes, activation='softmax')  # num_classes is the number of classes in your dataset

In [ ]:
model.add(base_model)

In [ ]:

# Load and preprocess the dataset
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    'C:/Users/Ali/Downloads/labs/lab1/fdataset/train',
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(224, 224),
    batch_size=32
)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    'C:/Users/Ali/Downloads/labs/lab1/fdataset/val',
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(224, 224),
    batch_size=32
)

# Configure the dataset for performance
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.cache().prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.cache().prefetch(buffer_size=AUTOTUNE)

# Compile the model
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

# Train the model
epochs = 10 # Adjust the number of epochs as needed
history = model.fit(train_dataset, epochs=epochs, validation_data=validation_dataset)

# Evaluate the model
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    'C:/Users/Ali/Downloads/labs/lab1/fdataset/train',
    seed=123,
    image_size=(224, 224),
    batch_size=32
)

model.evaluate(test_dataset)

